# Slå ihop datamängder med `merge`

*Av Anders Sundell*

Ofta när man gör statistiska analyser behöver man kombinera olika datamängder. Man kanske hämtar information om den oberoende variabeln från en källa, och vill samköra den med en beroende variabel från en annan källa. Vi har alltså samma analysenheter i de båda datamängderna, men vill kombinera variablerna. Då ska vi använda kommandot `merge`.

Ett annat scenario är deär man har olika datamängder med samma variabler, fast med olika analysenheter. Då ska man istället använda kommandot `append` - det är egentligen som att klistra in fler observationer längst ner i datamängden. Men eftersom `merge` är lite klurigare ska vi fokusera på det här.

## Olika typer av sammanslagningar

Med `merge`-kommandot kan vi göra tre (eller egentligen två) olika typer av sammanslagningar. Enklast är `1:1`. Då har vi samma analysenheter i båda datamängderna. Tex samma individer, eller samma länder, och vill lägga till ny information om dem. Vi behöver en matchningsvariabel som visar landets eller personens identitet så att vi kan matcha ihop dem.

Men vi kan också matcha `m:1`. Det betyder att vi har många analysenheter i datamängden som är öppen i Stata, och vill matcha på data från ett färre antal enheter på en högre nivå. Till exempel kanske vi har data på personer i Europa, och nu vill vi lägga till variabler på landsnivå. Vi kanske har gjort en enkätundersökning, och nu vill vi se hur svaren påverkas av om man bor i ett EU-land eller inte. Alla personer i Sverige kommer då få samma värde - vi bor i ett EU-land. Alla personer i Norge får också samma värde - de bor inte i ett EU-land. I vår individdatamängd kan vi då alltså ha tusentals personer, och kombinera det med en datamängd med et trettiotal observationer (en för varje land).

Nästan samma sak är att matcha `1:m`. Det är precis samma grej, men vi börjar då med att land-datamängden i Stata, och lägger till individdatan.


## Kombinera två mängder med samma typ av analysenheter - merge 1:1
Vi börjar med den enklaste varianten - när vi har samma typ av analysenheter i de båda datamängderna, och vill lägga till fler variabler. Säg att vi vill kombinera information om graden av demokrati i ett land, med graden av korruption i landet. Vi kan hämta information om demokratin från [Freedom House](https://freedomhouse.org/report-types/freedom-world). Datan finns i Excelformat, men det är bara att klippa ut de relevanta bitarna och lägga in i ett Stata-dataset. Sen hämtar vi information om korruption från [Transparency International](https://www.transparency.org/cpi2018), och gör på samma sätt där.

Vi har nu två dataset, `fh2017.dta` och `cpi2018.dta`. Nedan laddar vi in `fh2017.dta` och kollar på de fem första raderna i datamängden.

In [26]:
cd "/Users/anderssundell/Dropbox/Jupyter/stathelp/data"
use "fh2017.dta", clear
list in 1/5


/Users/anderssundell/Dropbox/Jupyter/stathelp/data



     +----------------------------------------+
     |     country   fh_pr   fh_cl   fh_sta~s |
     |----------------------------------------|
  1. | Afghanistan       5       6         NF |
  2. |     Albania       3       3         PF |
  3. |     Algeria       6       5         NF |
  4. |     Andorra       1       1          F |
  5. |      Angola       6       6         NF |
     +----------------------------------------+


Vi har alltså fyra variabler: landsnamnet `country`, graden av politiska rättigheter från 1-7 `fh_pr`, graden av medborgerliga rättigheter `fh_cl` och en sammanfattande beskrivning `fh_status`. Nu gör vi samma sak med korruptionsdatan.

In [27]:
use "cpi2018.dta", clear
list in 1/5




     +-----------------------+
     |     country   cpi2018 |
     |-----------------------|
  1. | Afghanistan        16 |
  2. |     Albania        36 |
  3. |     Algeria        35 |
  4. |      Angola        19 |
  5. |   Argentina        40 |
     +-----------------------+


Här har vi bara två variabler: landsnamnet `country` och graden av korruption `cpi2018` (där låga värden betyder mer korruption). Nu ska vi slå ihop de två datamängderna.

## Steg för steg

1. Se till att det finns en variabel i båda mängderna som det går att matcha på. I det här fallet är det `country`. Dvs, graden av demokrati i Afghanistan ska matchas ihop med graden av korruption i Afghanistan. Har man ingen sådan identifikationsvariabel går inte sammanslagningen att göra. Generellt sett är det dock att föredra om ID-variabeln är ett nummer, och inte en textvariabel. Det minskar risken för stavningsfel osv. Man kan också matcha på flera variabler - en vanlig tillämpning är att man matchar på både land och år. Dvs Sverige 2008 ska matchas ihop med Sverige 2008, osv.
2. Se till att matchningsvariabeln heter samma sak i båda datamängderna.
3. Öppna det ena datasetet. Nu har vi korruptionsdatan inladdad.
4. Ange koden för att slå ihop datamängderna, matchade på den relevanta variabeln.
5. Se hur det gick och åtgärda felaktigheter.

Vi använder kommandot `merge`. Strukturen är att man skriver `merge`, sedan vilken typ av matchning man vill göra, sedan vad matchningsvariabeln är, och sedan vilken datamängd man vill matcha ihop den öppna datan med. Vi vill matcha `1:1`, och göra det med demokratidatan. Det ser ut såhär:


In [28]:
merge 1:1 country using "fh2017.dta"


    Result                           # of obs.
    -----------------------------------------
    not matched                            50
        from master                        12  (_merge==1)
        from using                         38  (_merge==2)

    matched                               168  (_merge==3)
    -----------------------------------------


Nu är sammanslagningen klar. Men det finns felaktigheter att åtgärda. Tabellen visar att 168 observationer matchades framgångsrikt. Men vi kan också se att det var 50 enheter som inte kunde matchas: 12 från "master"-datamängden, dvs den vi hade inladdad först (dvs korruptionsdatan). Det var också 38 länder i "using"-datan, dvs demokratidatan, som inte kunda matchas ihop.

Ibland är det här inget större problem, tex om man matchar ihop data om EU-länder med data för hela världen. Då kommer det såklart inte gå att matcha ihop afrikanska länder med EU-datan.

Men här har vi möjliga problem, eftersom det finns länder i båda datamängderna som inte hittade någon match. Ofta beror det på att det är fel i identifikationsvariabeln.

## Identifiera missmatchade observationer
Som tur är hjälper Stata oss att lätt identifiera de felaktiga observationerna, eftersom det har skapats en ny variabel som heter `_merge`. Den visar resultatet av matchningen. Observationer med koden 1 fanns bara i den ursprungliga datan - observationer med värdet 2 fanns bara i den nya datan. Observationer med värdet 3 matchades ihop bra. Vi kan lista alla observationer som inte matchades, dvs alla som inte hade värdet 3:

In [29]:
list country _merge if _merge!=3


     +----------------------------------------------------+
     |                          country            _merge |
     |----------------------------------------------------|
 22. |                Brunei Darussalam   master only (1) |
 26. |                       Cabo Verde   master only (1) |
 36. |                            Congo   master only (1) |
 43. | Democratic Republic of the Congo   master only (1) |
 58. |                           Gambia   master only (1) |
     |----------------------------------------------------|
 66. |                    Guinea Bissau   master only (1) |
 70. |                        Hong Kong   master only (1) |
 85. |                     Korea, North   master only (1) |
 86. |                     Korea, South   master only (1) |
135. |                      Saint Lucia   master only (1) |
     |----------------------------------------------------|
136. | Saint Vincent and the Grenadines   master only (1) |
172. |         United States of America

Nästa steg är att se efter om det finns några länder som dyker upp i båda listorna. Ett slående exempel är USA. Vi ser att landet i ena datamängden heter "United States of America" och i andra "United States". På samma sät ser vi att Sydkorea heter "Korea, South" i ena datan och "South Korea" i den andra. När så är fallet kan Stata inte matcha ihop observationerna.

Vi måste därför rätta till datan så att identifikationsvariablerna stämmer överens. Vi går då in i datan och ändrar på variabeln - tex kan vi i demokratidatan byta från "United States" till "United States of America" och "Korea, South" till "South Korea". När vi sedan kör om vår dofil bör antalet omatchade länder ha minskat.

## `_merge`-variabeln
När vi är nöjda med matchningen kan vi ta bort data vi inte behöver. Till exempel den omatchade datan, eller observationer som inte fanns med i vår ursprungliga datamängd. Om vi till exempel vill ta bort observationer som bara fanns i demokratidatan skriver vi:

In [30]:
 drop if _merge==2

(38 observations deleted)


Sedan kan vi droppa `_merge`-variabeln. Det är ett måste om vi vill lägga till mer data - Stata protesterar om det redan finns en sådan. Därför skriver vi:

In [31]:
drop _merge

## Kombinera datamängder med analysenheter på olika nivåer - m:1

Nu kan vi istället tänka att vi har en datamängd med individer, som tex visar varje persons kön och dennes nöjdhet med demokratin. Jag har här hittat på lite data. Vi laddar in den och kollar på de 10 första personerna (det är bara 15 i hela datamängden):

In [32]:
use "satisfactiontest.dta", clear
list in 1/10




     +-------------------------------+
     |     country   sat_dem   woman |
     |-------------------------------|
  1. | Afghanistan         1       1 |
  2. | Afghanistan         3       0 |
  3. | Afghanistan         2       1 |
  4. | Afghanistan         4       0 |
  5. | Afghanistan         2       0 |
     |-------------------------------|
  6. |     Albania         5       1 |
  7. |     Albania         7       1 |
  8. |     Albania         5       0 |
  9. |     Albania         8       1 |
 10. |     Albania         4       0 |
     +-------------------------------+


Vi har alltså personer från Afghanistan och Albanien, som är olika nöjda med demokratin, och där vissa är kvinnor, och andra inte är det. Nu provar vi att matcha på demokratidatan med hjälp av `merge m:1`. Men det är fortfarande `country` som är matchningsvariabeln.

In [33]:
merge m:1 country using "fh2017.dta"


    Result                           # of obs.
    -----------------------------------------
    not matched                           203
        from master                         0  (_merge==1)
        from using                        203  (_merge==2)

    matched                                15  (_merge==3)
    -----------------------------------------


15 observationer matchades, som det var tänkt. 203 observationer matchades visserligen inte, men alla de återfanns i demokratidatan. Vi hade ju bara personer från 3 länder, så alla andra länder blir omatchade.

Nu tittar vi igen på de första 10 personerna:

In [35]:
list in 1/10


     +------------------------------------------------------------------------+
     |     country   sat_dem   woman   fh_pr   fh_cl   fh_sta~s        _merge |
     |------------------------------------------------------------------------|
  1. | Afghanistan         1       1       5       6         NF   matched (3) |
  2. | Afghanistan         3       0       5       6         NF   matched (3) |
  3. | Afghanistan         2       1       5       6         NF   matched (3) |
  4. | Afghanistan         4       0       5       6         NF   matched (3) |
  5. | Afghanistan         2       0       5       6         NF   matched (3) |
     |------------------------------------------------------------------------|
  6. |     Albania         5       1       3       3         PF   matched (3) |
  7. |     Albania         7       1       3       3         PF   matched (3) |
  8. |     Albania         5       0       3       3         PF   matched (3) |
  9. |     Albania         8       1   

Notera nu att alla variablerna från demokratidatan - `fh_pr`, `fh_cl`och `fh_status` har samma värde för alla personer från ett visst land. Dvs, alla personer från Afghanistan har samma värde på variabeln `fh_pr`, eftersom graden av politiska rättigheter är densamma för alla invånare i Afghanistan. Variabeln varierar bara på landsnivå.

Om vi hade börjat med att ha landsdatan inladdat i stata hade vi fått skriva `merge m:1 using "satisfactiontest.dta"` istället, men slutresultatet hade blivit detsamma.

## Att tänka på
1. Det måste finnas en matchningsvariabel som heter samma sak i båda datamängderna.
2. Matchningsvariablerna ska vara av samma typ. Om du själv klistrar in data från Excel till exempel, håll koll på att båda variablerna är numeriska om det är sifferkoder, eller att båda är textvariabler om det är namn. Man kan bland annat se det genom att öppna data editor och se vilken färg siffrorna har. Om de är svarta är det nummer. Om de är röda är de text.
3. För att man ska kunna lägga till variabler måste det vara samma observationer i båda datamängderna. Man kan till exempel inte kombinera data från olika enkätundersökningar med olika personer.
4. Ta bort onödiga observationer. Om du till exempel har ett huvuddataset och lägger till en variabel, ta bort alla onödiga observationer som följer med den nya variabeln. De kommer bara ställa till det i andra matchningar längre fram.
5. Kom ihåg att ta bort `_merge`-variabeln när du är färdig med matchningen, så att det går lätt att matcha på igen.
6. Var noggrann. Oftast när det blir fel med matchning är det på grund av att man skrivit in fel adress till datana och liknande.

## Avslutning
`Merge`-kommandot passar alltså bra när vi ska lägga till fler variabler. Vill vi lägga till observationer, som har information på samma variabler, ska vi istället använda `append`. Det skulle till exempel kunna vara att kombinera enkätundersökningar med identiska frågor som gjordes 2018 och 2019. Vi kan då med `append` lägga de olika enkätsvaren i en och samma datamängd.